In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math
from sklearn import preprocessing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
real_est = pd.read_csv('/content/drive/MyDrive/MachineLearning/datasets/Real_estate.csv')
real_est.head()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [4]:
#data normalization
data = real_est.iloc[:,[2,3,4,5,6,7]].values
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
data= min_max_scaler.fit_transform(data)

In [5]:
#X samples
X = data[:,:-1]
#y outcome
y = data[:,-1]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .8)
print('X train samples: ', len(X_train))
print('X test samples: ', len(X_test))

X train samples:  331
X test samples:  83


#KNN (K-Nearest Neighbors) from scratch

In [ ]:
#used to get the distances between the first input and every input in the dataset, including itself.
def euclidean_dist(x1,x2):
  distance = 0
  for i in range(len(x1)):
    distance += (x1[i]-x2[i])**2
  distance = np.sqrt(distance)
  return distance

#k: is the number of neighbors
def get_neighbors(X_train,x_i_test,y_train,k):
  dist_list = []
  for x_i_train,y_i_train in zip(X_train,y_train):
    dist = euclidean_dist(x_i_test,x_i_train)
    dist_list.append(([x_i_train,y_i_train], dist))
  #sort the list starting from the second element, i.e the distance
  dist_list.sort(key=lambda tup: tup[1])
  neighbors = []
  for k_i in range(k):
    neighbors.append(dist_list[k_i][0])

  #print('negih no dist', neighbors[0])
  return neighbors

#when KNN is used for regression we can take the mean or the median of the k-most similar istances
def get_mean_neigh(neighbors_out): 
  return np.mean(neighbors_out)

def RMSE(y_pred,y_true):
  RMSE = np.sqrt(np.mean((y_test - y_pred)**2))
  return RMSE

def KNN(X_train, X_test, y_train, y_test, k):
  preds = []
  for x_i_test in X_test:
    k_neighbors = get_neighbors(X_train,x_i_test,y_train,k)
    output_values = [row[-1] for row in k_neighbors]
    mean_out = get_mean_neigh(output_values)
    preds.append(mean_out)
  
  #compute RMSE
  rmse = RMSE(preds,y_train)
  print('RMSE', rmse)

In [ ]:
k=10
KNN(X_train, X_test, y_train, y_test, k)

RMSE 0.06566792709064512


*Comparison with the Sklearn model*

In [8]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

k=10
knn=KNeighborsRegressor(n_neighbors=k)
#train
model_knn=knn.fit(X_train,y_train)
#test
y_pred_sk=model_knn.predict(X_test)
mse=mean_squared_error(y_test,y_pred_sk)
print('RMSE sklearn KNN: ', np.sqrt(mse))

RMSE sklearn KNN:  0.06580985008907507
